# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,st. john's,47.5649,-52.7093,13.42,99,100,6.17,CA,1731466828
1,1,villa gesell,-37.2639,-56.9730,13.38,66,13,4.01,AR,1731466806
2,2,invercargill,-46.4000,168.3500,17.23,64,2,2.24,NZ,1731466931
3,3,bilibino,68.0546,166.4372,-26.38,99,39,0.77,RU,1731466932
4,4,ancud,-41.8697,-73.8203,10.06,94,76,2.13,CL,1731466815


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plt = city_data_df.hvplot.points("Lng", 
                                     "Lat", 
                                     size = "Humidity", 
                                     scale = .5,
                                     geo = True,
                                     tiles = "OSM",
                                     color = "City", 
                                     alpha = 0.5, 
                                     frame_width = 700, 
                                     frame_height = 500
                                    )

# Display the map
map_plt

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
#peronsal want 30 to 40 degrees Cels which should be high 60s to low 80s in fahren; windspeed 0-5m/s??; cloudiness??
city_data_df_narrowed = city_data_df.loc[(city_data_df["Max Temp"] >= 21 ) & (city_data_df["Max Temp"] <= 30) \
                            & (city_data_df["Humidity"] >= 75) & (city_data_df["Humidity"] <= 85) \
                            & (city_data_df["Cloudiness"] == 0) & (city_data_df["Cloudiness"] <= 22),:
                            ]
# vacation_spots_df
# Drop any rows with null values
city_data_df_narrowed = city_data_df_narrowed.dropna()

# Display sample data
city_data_df_narrowed

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
196,196,lom sak,16.7798,101.2422,26.94,78,0,1.03,TH,1731467088
236,236,ixtapa,20.7000,-105.2000,26.57,78,0,1.03,MX,1731467193


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df_narrowed[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotal Name"] = ""

# Display sample data
hotel_df 

,City,Country,Lat,Lng,Humidity,Hotal Name
196,lom sak,TH,16.7798,101.2422,78,
236,ixtapa,MX,20.7000,-105.2000,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = { 
    "categories": "accommodation.hotel", 
    "apiKey": geoapify_key, 
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    print(name_address.status_code)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

# print(name_address)

Starting hotel search
200
lom sak - nearest hotel: Sawang Hotel
200
ixtapa - nearest hotel: Ma. Cristina


,City,Country,Lat,Lng,Humidity,Hotal Name,Hotel Name
196,lom sak,TH,16.7798,101.2422,78,,Sawang Hotel
236,ixtapa,MX,20.7000,-105.2000,78,,Ma. Cristina


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plt = hotel_df.hvplot.points("Lng",
                               "Lat",
                               size = "Humidity",
                               scale = 1,
                               geo = True,
                               tiles = "OSM",
                               color = "City",
                               alpha = 1,
                               frame_width = 700,
                               frame_height = 500,
                               hover_cols = ["Hotel Name", "Country"]
                               )

# Display the map
map_plt

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)